In [7]:
import clip
import torch
from PIL import Image
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Load the model: CLIP

In [2]:

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {device} device")

# Load the model
model, preprocess = clip.load('ViT-B/32', device=device)

Using cuda device


In [58]:
# Load an image

image_paths = ['../images/5536532658_3_1_1.jpg', '../images/5536532658_6_1_1.jpg', '../images/5767521712_3_1_1.jpg', '../images/81_0_2024_V_0_3.jpeg']

images = [preprocess(Image.open(image_path)).unsqueeze(0).to(device) for image_path in image_paths]

In [59]:

with torch.no_grad():
    image_features_l = [model.encode_image(image) for image in images]

image_features = torch.stack(image_features_l).squeeze()

In [60]:
image_features.shape

torch.Size([4, 512])

In [25]:
# Compute distances between similar and not

cosine_similarity_same = torch.nn.functional.cosine_similarity(image_features_l[0], image_features_l[1])

cosine_similarity_diff = torch.nn.functional.cosine_similarity(image_features_l[0], image_features_l[2])

print(f"Similarity between image 1 and 2 (same): {cosine_similarity_same.item()}")

print(f"Similarity between image 1 and 3 (different): {cosine_similarity_diff.item()}")

Similarity between image 1 and 2 (same): 0.95849609375
Similarity between image 1 and 3 (different): 0.7919921875


In [27]:
image_paths = ['../images/11_0_2024_V_0_1.jpeg', '../images/11_1_2024_V_0_1.jpeg', '../images/11_2_2024_V_0_1.jpeg']

images = [preprocess(Image.open(image_path)).unsqueeze(0).to(device) for image_path in image_paths]

with torch.no_grad():
    image_features = [model.encode_image(image) for image in images]
    
    
cosine_similarity_1_2 = torch.nn.functional.cosine_similarity(image_features[0], image_features[1])
cosine_similarity_2_3 = torch.nn.functional.cosine_similarity(image_features[1], image_features[2])
cosine_similarity_1_3 = torch.nn.functional.cosine_similarity(image_features[0], image_features[2])



print(f"Similarity between image 1 and 2: {cosine_similarity_1_2.item()}")

print(f"Similarity between image 2 and 3: {cosine_similarity_2_3.item()}")

print(f"Similarity between image 1 and 3: {cosine_similarity_1_3.item()}")

Similarity between image 1 and 2: 0.912109375
Similarity between image 2 and 3: 0.826171875
Similarity between image 1 and 3: 0.75


In [29]:
image_paths = ['../images/models/96_0_2024_V_0_1', '../images/models/96_2_2024_V_0_1', '../images/models/100_2_2023_I_0_1']

images = [preprocess(Image.open(image_path)).unsqueeze(0).to(device) for image_path in image_paths]

with torch.no_grad():
    image_features = [model.encode_image(image) for image in images]
    
    
cosine_similarity_1_2 = torch.nn.functional.cosine_similarity(image_features[0], image_features[1])
cosine_similarity_2_3 = torch.nn.functional.cosine_similarity(image_features[1], image_features[2])
cosine_similarity_1_3 = torch.nn.functional.cosine_similarity(image_features[0], image_features[2])



print(f"Similarity between image 1 and 2: {cosine_similarity_1_2.item()}")

print(f"Similarity between image 2 and 3: {cosine_similarity_2_3.item()}")

print(f"Similarity between image 1 and 3: {cosine_similarity_1_3.item()}")

Similarity between image 1 and 2: 0.64453125
Similarity between image 2 and 3: 0.78759765625
Similarity between image 1 and 3: 0.576171875


In [75]:
tabular_data = np.array([
    ['m', 's', 'v'],
    ['m', 's', 'v'],
    ['m', 'd', 'v'],
    ['m', 's', 'v'],
])

weight = 10.0

In [76]:
encoder = OneHotEncoder()
encoded_categorical = encoder.fit_transform(tabular_data).toarray()
encoded_categorical = torch.tensor(encoded_categorical, device='cuda').float()

In [77]:
combined_features = torch.cat((image_features, weight*encoded_categorical), dim=1)

In [78]:
similarity_score = torch.nn.functional.cosine_similarity(combined_features[0].unsqueeze(0), combined_features[1].unsqueeze(0))
print("Cosine Similarity:", similarity_score.item())

Cosine Similarity: 0.9890365600585938


In [79]:
similarity_score = torch.nn.functional.cosine_similarity(combined_features[0].unsqueeze(0), combined_features[2].unsqueeze(0))
print("Cosine Similarity:", similarity_score.item())

Cosine Similarity: 0.7012704610824585


In [80]:
similarity_score = torch.nn.functional.cosine_similarity(combined_features[0].unsqueeze(0), combined_features[3].unsqueeze(0))
print("Cosine Similarity:", similarity_score.item())

Cosine Similarity: 0.9810919165611267


: 